In [4]:
import exifread as EXIF
import os
import sys
from pykml.factory import KML_ElementMaker as KML
from lxml import etree

#Modified from https://developers.google.com/kml/articles/geotagsimple

#Still need to test on actually geotagged photos
# ipad photos apparently need to be exported from iPhoto (only on Mac) and then checked to export geotags too. Otherwise they're stripped. 

# Do the following for each
# Append to .kml file 
#
# Copyright 2008 Google Inc. All Rights Reserved.
u"""Reads the EXIF headers from geo-tagged photos. and creates a KML file.
Reads the EXIF headers from geo-tagged photos and creates a KML file with
a PhotoOverlay element for each file. Requires the open source EXIF.py file
downloadable at:
http://sourceforge.net/projects/exif-py/
   GetFile(): Handles the opening of an individual file.
  GetHeaders(): Reads the headers from the file.
  DmsToDecimal(): Converts EXIF GPS headers data to a decimal degree.
  GetGps(): Parses out the the GPS headers from the headers data.
  CreateKmlDoc(): Creates an XML document object to represent the KML document.
  CreatePhotoOverlay: Creates an individual PhotoOverlay XML element object.
  CreateKmlFile(): Creates and writes out a KML document to file.

  __author__ = 'mmarks@google.com (Mano Marks)'

"""

u"Reads the EXIF headers from geo-tagged photos. and creates a KML file.\nReads the EXIF headers from geo-tagged photos and creates a KML file with\na PhotoOverlay element for each file. Requires the open source EXIF.py file\ndownloadable at:\nhttp://sourceforge.net/projects/exif-py/\n   GetFile(): Handles the opening of an individual file.\n  GetHeaders(): Reads the headers from the file.\n  DmsToDecimal(): Converts EXIF GPS headers data to a decimal degree.\n  GetGps(): Parses out the the GPS headers from the headers data.\n  CreateKmlDoc(): Creates an XML document object to represent the KML document.\n  CreatePhotoOverlay: Creates an individual PhotoOverlay XML element object.\n  CreateKmlFile(): Creates and writes out a KML document to file.\n\n  __author__ = 'mmarks@google.com (Mano Marks)'\n\n"

In [13]:
def GetFile(file_name):
    """Handles opening the file.
    Args:
    file_name: the name of the file to get
    Returns:
    A file
    """

    the_file = None
    try:
        the_file = open(file_name, 'rb')
    except IOError:
        the_file = None
    return the_file



def GetHeaders(the_file):
    """Handles getting the EXIF headers and returns them as a dict.
    Args:
    the_file: A file object
    Returns:
    a dict mapping keys corresponding to the EXIF headers of a file.
    """
    data = EXIF.process_file(the_file)  # , 'UNDEF', False, False, False
    return data



def DmsToDecimal(degree_num, degree_den, minute_num, minute_den,
                 second_num, second_den):
    """Converts the Degree/Minute/Second formatted GPS data to decimal degrees.
    Args:
    degree_num: The numerator of the degree object.
    degree_den: The denominator of the degree object.
    minute_num: The numerator of the minute object.
    minute_den: The denominator of the minute object.
    second_num: The numerator of the second object.
    second_den: The denominator of the second object.
    Returns:
    A deciminal degree.
    """
    degree = float(degree_num)/float(degree_den)
    minute = float(minute_num)/float(minute_den)/60
    second = float(second_num)/float(second_den)/3600
    return degree + minute + second

def GetGps(data):
    """Parses out the GPS coordinates from the file.
    Args:
    data: A dict object representing the EXIF headers of the photo.
    Returns:
    A tuple representing the latitude, longitude, and altitude of the photo.
    """
    lat_dms = data['GPS GPSLatitude'].values
    long_dms = data['GPS GPSLongitude'].values
    latitude = DmsToDecimal(lat_dms[0].num, lat_dms[0].den,
                            lat_dms[1].num, lat_dms[1].den,
                            lat_dms[2].num, lat_dms[2].den)

    longitude = DmsToDecimal(long_dms[0].num, long_dms[0].den,
                            long_dms[1].num, long_dms[1].den,
                            long_dms[2].num, long_dms[2].den)
    # Error here handled earlier, in main loop
    if data['GPS GPSLatitudeRef'].printable == 'S': latitude *= -1
    if data['GPS GPSLongitudeRef'].printable == 'W': longitude *= -1
    altitude = None
    try:
        alt = data['GPS GPSAltitude'].values[0]
        altitude = alt.num/alt.den
        if data['GPS GPSAltitudeRef'] == 1: altitude *= -1
    except KeyError:
        altitude = 0
    return latitude, longitude, altitude

In [14]:
##Make KML with pykml using https://pythonhosted.org/pykml/examples/kml_reference_examples.html
def CreateKmlDoc(kmlname, usr_desc):
    kml_kml = KML.kml()

    kml_doc = KML.Document(
        KML.name(kmlname),
        KML.description(usr_desc)
        )
    return kml_kml, kml_doc
    
def CreatePhotoOverlay(kml_doc, file_name, the_file, file_iterator):
    photo_id = 'photo{}'.format(file_iterator)
    data = GetHeaders(the_file)

    coords = GetGps(data)
    
    # Determines the proportions of the image and uses them to set FOV.
    width = float(data['EXIF ExifImageWidth'].printable)
    length = float(data['EXIF ExifImageLength'].printable)
    lf = str(width/length * -20.0)
    rf = str(width/length * 20.0)
    
    PO = KML.PhotoOverlay(
        KML.name(file_name),
        KML.id(photo_id),
        KML.Camera(
            KML.longitude(coords[1]),
            KML.latitude(coords[0]),
            KML.altitude(10),
            KML.tilt(90)
            ),
        KML.styleUrl('camera'),
        KML.Icon(
            KML.href(file_name)    #usr_path + '\\'+ file_name
        ),
        KML.ViewVolume(
            KML.near(50),
            KML.leftFov(lf),
            KML.rightFov(rf),
            KML.bottomFov(-20),
            KML.topFov(20)
        ),
        KML.Point(
            KML.coordinates('{},{},{}'.format(coords[1], coords[0], coords[2]))
        )
    )
    
    kml_doc.append(PO)

In [21]:
# Read files in directory
# Input necessary (usrcode == 1), input hardcoded (Adv Geomorph) (2)
usrcode = 1

if usrcode ==1: 
    usr_path = raw_input("Enter directory path:")
    the_file = str(usr_path)
else: 
    usr_path = 'C:\Users\\nariv\OneDrive\Research\Landuse\Code\Photos_to_KML\AdvGeomorphFieldTrip'
    usr_desc = u'Adv Geomorph 2017 visits Seneca Falls, \n \
                the Salt River, \n \
                Rim gravels preserved beneath basalt flows, exposed in roadcut \n \
                Whiteriver, more basalts (some on hilltops, some in the valley) preserving Rim Gravels \n \
                Pinetop (dinner), \n \
                the 260 Overlook from the Rim -- Main question: Is the topographic relief due to erosion, or tectonics? \n \
                Bear Flat Road and Tonto Creek -- investigating the knickpoint in the Tonto Creek, and the low-relief terrain transitioning to a canyon, \n \
                more gravels (sandier) between Pine and Payson - younger gravels with dominant flow direction to the East, topped by basalt, \n \
                overlook of the Rye Valley - multiple stages of filling and emptying the basin-and-range topography \
                in the relatively tectonically quiet times the past ~5Ma,\n \
                and a roadcut of an andesite flow (~23 Ma) topped by cobbles and valley fill.'        

photo_counter = 0
filenames = []

for root, dirs, files in os.walk(usr_path):
    for f in files:
        extension = os.path.splitext(f)[1]
        print extension
        if extension == '.png' or extension == '.jpg':
            filenames.append(f)
    kmlname = root.split('\\')[-1]
    fullkmlname = '{}\{}.kml'.format(root,kmlname)
print filenames, kmlname

Enter directory path:C:\Users\nvmille1\OneDrive\Research\Landuse\Field_2016\July_21
.JPG
.JPG
.JPG
.JPG
[] Profile_1


In [16]:
file_iterator = 1
kml_kml, kml_doc = CreateKmlDoc(fullkmlname, kmlname)

for f in filenames:
    photo_counter += 1
    the_file = GetFile(usr_path + '\\' + f)
    if the_file is None:
        print "'{}' is unreadable\n".format(f)
        filenames.remove(f)
        continue
    tdata = GetHeaders(the_file)
    try: 
        tdata['GPS GPSLatitudeRef']
    except KeyError:
        print"'{}' has no GPS GPSLatitudeRef".format(f)
        filenames.remove(f)
        continue
        
    CreatePhotoOverlay(kml_doc, f, the_file, file_iterator)
    file_iterator += 1
    the_file.close()
    
#After all photo overlays have been written, append the doc to the outermost layer, kml_kml
kml_kml.append(kml_doc)
  
fld_contents = etree.tostring(kml_kml, pretty_print=True)    
kml_file = open(fullkmlname, 'w')
kml_file.write(fld_contents)
kml_file.close()


print 'Finished writing {} photo overlays to {}.'.format(file_iterator, kmlname)

'8ACE7B73EC7B3C51BD24667A2A80BE66.pdf' has no GPS GPSLatitudeRef
'Additional Notes.docx' has no GPS GPSLatitudeRef
'ArcMMTpts-test.xlsx' has no GPS GPSLatitudeRef
'Field_2016_SampleSheet.xlsx' has no GPS GPSLatitudeRef


KeyError: 'GPS GPSLatitude'

In [7]:
#Option to make kmz file: 
#Need to ask, make file structure, then: 
usr_kmz = raw_input("Make KMZ? (y/n):")

if usr_kmz == str(n):
	sys.exit()


from zipfile_infolist import print_info
import zipfile

zipfile.ZipFile('Photos.kmz', mode= 'w')
zf.write('/Images')

if CreateKMZ = True:
	photo_counter = 0

	for root, dirs, filenames in os.walk(indir):
		for f in filenames:
			photo_counter += 1

			print 'creating archive'
			zf = zipfile.ZipFile('zipfile_write.zip', mode='a')
			try:
			    print 'photo #{}'.format(photo_counter)
			    zf.write('~/Images/{}'.format(f))
			finally:
			    print 'closing'
			    zf.close()

			print
			print_info('zipfile_write.zip')

SyntaxError: invalid syntax (<ipython-input-7-621d79689d14>, line 15)